In [2]:
import pandas as pd
import numpy as np
import random as rand
import tqdm
import networkx as nx
import pickle as pkl
import gower

In [8]:
with open('graph_349519.pkl', 'rb') as input:
    graph = pkl.load(input)

# Extracting data
edge_index = graph['edge_index']
num_nodes = graph['num_nodes']

# Create a networkx graph
G = nx.Graph()

# Add nodes
G.add_nodes_from(range(num_nodes))

# Add edges
edges = list(zip(edge_index[0], edge_index[1]))
G.add_edges_from(edges)

In [116]:
# Create a lookup table for gower distances
attributes = graph['node_feat']
lu = pd.DataFrame(attributes)

In [389]:
def prototype(G):
    prot = rand.choice([n for n in G])
    return prot

def getAttributes(x):
    return [float(i) for i in attributes[x]]

def ranking(x, S: nx.Graph):
    D = nx.shortest_path_length(S, x)
    distances = list(D.values())
    D = list(zip(D.keys(),D.values()))


    for d in np.unique(distances):
        left, right = distances.index(d), len(distances) - distances[::-1].index(d)
        tielist = D[left:right]
        gowerlist = gower.gower_matrix(pd.DataFrame([getAttributes(x)]), pd.DataFrame([getAttributes(i[0]) for i in tielist]))
        sorted_gower = [x for _,x in sorted(zip(gowerlist[0],tielist))]
        D[left:right] = sorted_gower

    return D

def Q(S, G, target):
    S_size = len(S)
    G_size = len(G)
    cover = S_size / G_size
    
    n_target_S = lu.loc[S][target].sum()
    n_target_G = lu.loc[G][target].sum()

    
    WRAcc = (cover**0.5)* ((n_target_S/S_size) - (n_target_G/G_size))
    RAcc = ((n_target_S/S_size) - (n_target_G/G_size))

    return abs(WRAcc)

def Discovery(ranks):
    rho = 0
    sigma = 0
    best = 0

    for i in range(1,len(ranks)):
        q = Q([x[0] for x in ranks[0:i]], list(G.nodes), 'target')
        if q >= best:
            best = q
            rho = i
    
    best = 0

    for i in range(1,rho):
        q = Q([x[0] for x in ranks[0:i]], [x[0] for x in ranks[0:rho]], 'target')
        if q >= best:
            best = q
            sigma = i

    return rho, sigma, best, ranks

In [380]:
lu['target'] = lu[0] >= 6

In [383]:
out = pd.DataFrame(columns= ['rho', 'sigma', 'q', 'ranks'])
for node in tqdm.tqdm(list(G.nodes)):
    rho, sigma, q, ranks = Discovery(ranking(prototype(G), G))
    out.loc[node] = [rho, sigma, q, ranks]

100%|██████████| 332/332 [01:56<00:00,  2.86it/s]


In [391]:
out.sort_values(by=['q'], ascending= False).iloc[0]

rho                                                      3
sigma                                                    1
q                                                   0.3849
ranks    [(147, 0), (149, 1), (148, 1), (141, 1), (140,...
Name: 272, dtype: object

In [47]:
pd.DataFrame([getAttributes(prototype(G))])

,0,1,2,3,4,5,6,7,8
0,7,0,2,5,1,0,2,0,0
